In [0]:
tiers= ["bronze","silver","gold"]

dbfs_path= { tier : f"/Volumes/workspace/eathquake_data/raw-data/{tier}" for tier in tiers}

bronze_dbfs= dbfs_path["bronze"]
silver_dbfs= dbfs_path["silver"]
gold_dbfs= dbfs_path["gold"]

dbutils.fs.ls(bronze_dbfs)
dbutils.fs.ls(silver_dbfs)
dbutils.fs.ls(gold_dbfs)
# print("Hello Earthquake")
# print("Hello Earthquake Url")

In [0]:
import requests
import json
from datetime import date, timedelta

In [0]:
start_date= date.today() - timedelta(1)
end_date= date.today()
print(f" { start_date} and end date is { end_date}")

In [0]:
url= f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={start_date}&endtime={end_date}"

try:
    response= requests.get(url)
    #check if request was succsfull or not
    response.raise_for_status()
    data= response.json().get('features',[])
    if not data :
        print("no date returned for specific data range")
    else:
        # specify the file path
        file_path= f"{bronze_dbfs}/{start_date}_earthquake_data.json"
        json_data= json.dumps(data,indent=4)
        dbutils.fs.put(file_path, json_data, overwrite=True)
        print(f"Data successfully saved to {file_path}")
except requests.exceptions.RequestException as e:
    print(f"Error fetching the data from API : {e}")






In [0]:
output_data= {
    "start_date": start_date.isoformat(),
    "end_date": end_date.isoformat(),
    "bronze_dbfs" : bronze_dbfs,
    "silver_dbfs" : silver_dbfs,
    "gold_dbfs" : gold_dbfs

}
dbutils.jobs.taskValues.set(key="bronze_output", value=output_data)